In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/celeba-dataset/list_attr_celeba.csv")
extracted = pd.read_csv("../input/extracted/face_align.csv")
print("shape of data>",data.shape)
print("shape of extracted>", extracted.shape)

import cv2
face_classifier = cv2.CascadeClassifier('../input/opencvhaarcascades/Haarcascades/haarcascade_frontalface_default.xml')<br>
face_align = np.zeros((dataset.shape[0], 4), dtype = np.int16)<br>

### an array on all the images and extracting faces from the image
for i in range(dataset.shape[0]):<br>
    position = dataset['image_id'][i]<br>
    curr_image = cv2.imread("../input/celeba-dataset/img_align_celeba/img_align_celeba/"+position)<br>
    
    face = face_classifier.detectMultiScale(curr_image, 1.3,5)<br>
    
    # if facial dimension are found in the image
    if face is not ():<br>
        x,y,w,h = face[0]<br>
        face_align[i,0] = x<br>
        face_align[i,1] = y<br>
        face_align[i,2] = w<br>
        face_align[i,3] = h<br>

In [ ]:
data = data[['image_id', 'Male']]
data = data.replace(-1,0)

## above is data

In [ ]:
dataset = data.iloc[150000:200000,:].values
face_info = extracted.iloc[150000:200000,:].values

In [ ]:
from sklearn.utils import shuffle
dataset, face_info = shuffle(dataset, face_info, random_state=32)

## Template
x,y,w,h = face_info[['x','y','w','h']].iloc[1].values<br>
print(x,y,w,h)<br>
img = cv2.imread("../input/celeba-dataset/img_align_celeba/img_align_celeba/"+dataset.iloc[0,0],0)<br>
plt.imshow(img[int(y):int(y)+int(h),int(x):int(x)+int(w)])

In [ ]:
dataX = []
dataY = []

In [ ]:
for i in range(dataset.shape[0]):
    x,y,w,h = face_info[i,:]
    
    if w != 0 and h != 0:
        img = cv2.imread("../input/celeba-dataset/img_align_celeba/img_align_celeba/"+dataset[i,0],0)
        dataX.append(cv2.resize(img[int(y):int(y)+int(h),int(x):int(x)+int(w)] , (112,112)))
        dataY.append(dataset[i,1])


In [ ]:
nx = len(dataX)
ny = len(dataY)
print(nx,ny)

In [ ]:
X_train = np.array(dataX[:int(nx*0.9)]).reshape(int(nx*0.9), 112,112,1)
Y_train = dataY[:int(nx*0.9)]

X_val = np.array(dataX[int(nx*0.9):]).reshape(nx-int(nx*0.9), 112,112,1)

Y_val = dataY[int(nx*0.9):]


In [ ]:
from keras import optimizers
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from time import time


################### neural network ######################
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation="relu", input_shape=(112,112,1)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(loss="mean_squared_error", optimizer=optimizers.Adam(learning_rate=0.00001), metrics=['acc'])

In [ ]:
model.load_weights("model_weights3.h5")

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


BATCH_SIZE = 32
train_generator = train_datagen.flow(X_train, Y_train, batch_size = BATCH_SIZE)
test_generator = test_datagen.flow(X_val, Y_val, batch_size=BATCH_SIZE)

In [ ]:
ntrain = X_train.shape[0]
nval = X_val.shape[0]
history = model.fit_generator(train_generator, steps_per_epoch = ntrain//BATCH_SIZE,
                              epochs = 64,
                              validation_data = test_generator,
                              validation_steps = nval//BATCH_SIZE)

In [ ]:
model.save_weights('model_weights4.h5')

#model.save('model_keras.h5')


In [ ]:
model.save('model_keras.h5')

In [ ]:
data = pd.DataFrame(history.history)

data.head()

In [ ]:
data.to_csv("hist4.csv")

In [ ]:
del  data, extracted, dataX, dataY, X_train, X_val, Y_train, Y_val